### **INSTALL AND IMPORT LIBRARIES**

In [1]:
import sqlite3
import pandas as pd

### **QUERY THE DATA FROM DATABASE**

In [2]:
# Load the SQL file into a SQLite database
conn = sqlite3.connect('../data/raw/metadata.db')  # Use an in-memory database
cursor = conn.cursor()

# Query the data
cursor.execute("SELECT rec_id, url, website, result, created_date FROM `index`")  # Escape the table name
data = cursor.fetchall()

# Close the connection
conn.close()

# Convert to a pandas DataFrame for easier manipulation
df = pd.DataFrame(data, columns=['rec_id', 'url', 'website', 'result', 'created_date'])
df.drop_duplicates(inplace=True)
df

,rec_id,url,website,result,created_date
0,1,http://intego3.info/EXEL/index.php,1613573972338075.html,1,2021-02-17 20:29:32
1,2,https://www.mathopenref.com/segment.html,1635698138155948.html,0,2021-10-31 16:35:38
2,3,https://www.computerhope.com/issues/ch000254.htm,1635699228889266.html,0,2021-10-31 16:53:48
3,4,https://www.investopedia.com/terms/n/next-elev...,1635750062162701.html,0,2021-11-01 12:31:02
4,5,https://jobs.emss.org.uk/lcc.aspx,161356510250721.html,0,2021-02-17 18:01:42
...,...,...,...,...,...
79995,79996,https://bestjobmanage.com/adminservicedesk/True/,1622125898152592.html,1,2021-05-27 14:31:38
79996,79997,http://vieuxshack.com/download/adobe/b51f18074...,160822961118687.html,1,2020-12-17 23:56:51
79997,79998,https://www.tumblr.com/search/gas%20mask%20tattoo,163570726902772.html,0,2021-10-31 19:07:49
79998,79999,https://www.magnetic-shield.com/pdf/wc_4.pdf,1635701992217159.html,0,2021-10-31 17:39:52


### **REDUCE AND BALANCE THE DATA**

In [3]:
# Sample each class
sample = 30000
df_0 = df[df['result'] == 0].sample(n=sample, random_state=42) # Legitimate
df_1 = df[df['result'] == 1].sample(n=sample, random_state=42) # Phishing

# Combine the sampled data
df_balanced = pd.concat([df_0, df_1])

# Shuffle the final dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
df_balanced

,rec_id,url,website,result,created_date
0,62854,https://docs.embotics.com/Service-Portal/vm_sn...,1613531218856905.html,0,2021-02-17 08:36:58
1,70378,https://csgocyber.ru/freeskin,1621888236174848.html,1,2021-05-24 20:30:36
2,63608,https://1stglobalcapitalinvestors.com/linkedin...,1613575467981084.html,1,2021-02-17 20:54:27
3,64650,https://help.foxnews.com/hc/en-us/sections/206...,1607254628655677.html,0,2020-12-06 17:07:08
4,53403,https://www.calculatorsoup.com/calculators/mat...,163570766663381.html,0,2021-10-31 19:14:26
...,...,...,...,...,...
59995,37939,Https://viabcp.moaqt.com,161357270863021.html,1,2021-02-17 20:08:28
59996,13566,http://bonprixsklep.com.pl/glosowanie,1607278209044228.html,1,2020-12-06 23:40:09
59997,41726,https://www.sbs.com.au/,1613568353531711.html,0,2021-02-17 18:55:53
59998,48414,https://www.analystforum.com/t/type-i-vs-type-...,1635713509174732.html,0,2021-10-31 20:51:49


### **EXPORT THE DATA**

In [4]:
# Drop unnecessary columns and duplicates
df_final = df_balanced.drop(columns=['rec_id', 'created_date']).reset_index(drop=True)
df_final = df_final.drop_duplicates()
df_final.to_csv('../data/processed/dataset_full.csv', index=False)

### **CREATE AND EXPORT EXTRA DATA**

In [5]:
# 200 phishing samples from df_balanced
phish_extra = df_balanced[df_balanced['result'] == 1]
phish_extra = phish_extra.sample(n=200, random_state=42)

# 200 Legitimate samples not in df_balanced
legit_extra = df[~df['url'].isin(df_balanced['url']) & (df['result'] == 0)]
legit_extra = legit_extra.sample(n=200, random_state=42)

# Combine and save
df_extra = pd.concat([phish_extra, legit_extra]).sample(frac=1, random_state=42)
df_extra = df_extra.drop(columns=['rec_id', 'created_date']).reset_index(drop=True)
df_extra.to_csv('../data/processed/dataset_extra.csv', index=False)